<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [39]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [40]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""
with open('./data/hopsworks-api-key.txt', 'r', encoding='utf-8') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read()

project = hopsworks.login()
fs = project.get_feature_store()
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1149079
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "france", "city": "paris", "street": "18eme", "aqicn_url": "https://api.waqi.info/feed/@3082", "latitude": 48.85, "longitude": 2.35}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [41]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [42]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,71.0,france,paris,18eme,2024-11-16,https://api.waqi.info/feed/@3082


In [43]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [44]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 48.75°N 2.25°E
Elevation 46.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-16,9.70,0.0,2.160000,270.000000,paris
1,2024-11-17,10.85,0.0,10.296251,306.469330,paris
2,2024-11-18,11.45,0.0,20.630230,240.751266,paris
3,2024-11-19,12.40,0.1,26.865265,237.587708,paris
4,2024-11-20,6.35,0.0,21.068459,289.983185,paris
5,2024-11-21,4.10,0.0,9.585739,214.286926,paris
6,2024-11-22,7.20,0.0,21.915693,284.264557,paris
7,2024-11-23,5.55,0.2,23.795427,190.459854,paris
8,2024-11-24,14.60,0.0,19.110542,183.239639,paris
9,2024-11-25,15.65,0.2,31.614477,210.068497,paris


In [45]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


In [46]:
daily_df.head(5)

,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-16,9.70,0.0,2.160000,270.000000,paris
1,2024-11-17,10.85,0.0,10.296251,306.469330,paris
2,2024-11-18,11.45,0.0,20.630230,240.751266,paris
3,2024-11-19,12.40,0.1,26.865265,237.587708,paris
4,2024-11-20,6.35,0.0,21.068459,289.983185,paris


In [47]:
aq_today_df.head(5)

,pm25,country,city,street,date,url
0,71.0,france,paris,18eme,2024-11-16,https://api.waqi.info/feed/@3082


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [48]:
air_quality_fg

In [49]:
aq_today_df

,pm25,country,city,street,date,url
0,71.0,france,paris,18eme,2024-11-16,https://api.waqi.info/feed/@3082


In [50]:
# Insert new data
air_quality_fg.insert(aq_today_df)
aq_today_df['pm25_3days'] = air_quality_fg['pm25'].shift(1).rolling(window=3, min_periods=1).mean()
aq_today_df.head()

FeatureStoreException: Features are not compatible with Feature Group schema: 
 - pm25_3days (type: 'double') is missing from input dataframe.
Note that feature (or column) names are case insensitive and spaces are automatically replaced with underscores.

In [ ]:
air_quality_fg.insert(aq_today_df)

In [ ]:
# Insert new data
weather_fg.insert(daily_df)

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
